In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
KEY=os.getenv("OPENAI_API_KEY")

In [6]:
from langchain_openai import ChatOpenAI

In [7]:
llm = ChatOpenAI(openai_api_key=KEY, temperature=0.5)

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [12]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path=r"D:\Data Science\large language models\mcqgen\data.txt"

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [22]:
print(TEXT)

Linear Regression
Theory: At its core, linear regression is a method to predict the value of a variable based on the value of one or more other variables. The prediction is made under the assumption that the relationship between these variables is linear, meaning that if you were to plot them on a graph, they would roughly align along a straight line. The goal of linear regression is to find the best-fitting line through the data points that minimizes the differences between the predicted values and the actual values. This best-fitting line is then used to make predictions.

Logistic Regression
Theory: Logistic regression, unlike linear regression, is used when the outcome we are trying to predict is categorical, typically a yes/no outcome. Instead of fitting a straight line to the data, logistic regression fits a curve that estimates the probability of the occurrence of a yes (or 1) outcome. The theory behind logistic regression is to model the odds of the dependent variable as a line

In [23]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [25]:
NUMBER=5 
SUBJECT="artificial intelligence"
TONE="simple"

In [26]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

d:\Data Science\large language models\mcqgen\env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Linear Regression
Theory: At its core, linear regression is a method to predict the value of a variable based on the value of one or more other variables. The prediction is made under the assumption that the relationship between these variables is linear, meaning that if you were to plot them on a graph, they would roughly align along a straight line. The goal of linear regression is to find the best-fitting line through the data points that minimizes the differences between the predicted values and the actual values. This best-fitting line is then used to make predictions.

Logistic Regression
Theory: Logistic regression, unlike linear regression, is used when the outcome we are trying to predict is categorical, typically a yes/no outcome. Instead of fitting a straight line to the data, logistic regression fits a curve that estimates the probability of the occurrence of a yes (o

In [27]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2332
Prompt Tokens:1520
Completion Tokens:812
Total Cost:0.0039040000000000004


In [28]:
response

{'text': 'Linear Regression\nTheory: At its core, linear regression is a method to predict the value of a variable based on the value of one or more other variables. The prediction is made under the assumption that the relationship between these variables is linear, meaning that if you were to plot them on a graph, they would roughly align along a straight line. The goal of linear regression is to find the best-fitting line through the data points that minimizes the differences between the predicted values and the actual values. This best-fitting line is then used to make predictions.\n\nLogistic Regression\nTheory: Logistic regression, unlike linear regression, is used when the outcome we are trying to predict is categorical, typically a yes/no outcome. Instead of fitting a straight line to the data, logistic regression fits a curve that estimates the probability of the occurrence of a yes (or 1) outcome. The theory behind logistic regression is to model the odds of the dependent vari

In [30]:
quiz = response.get("quiz")

In [32]:
quiz=json.loads(quiz)

In [33]:
quiz

{'1': {'mcq': 'What is the goal of linear regression?',
  'options': {'a': 'To find the best-fitting line through the data points',
   'b': 'To predict the value of a variable based on other variables',
   'c': 'To estimate the probability of a categorical outcome',
   'd': 'To understand and interpret human language'},
  'correct': 'a'},
 '2': {'mcq': 'When is logistic regression used?',
  'options': {'a': 'When the outcome to predict is categorical',
   'b': 'When the relationship between variables is linear',
   'c': 'When processing and analyzing natural language data',
   'd': 'When generating coherent and contextually relevant text'},
  'correct': 'a'},
 '3': {'mcq': 'What is the main goal of Natural Language Processing (NLP)?',
  'options': {'a': 'To find the best-fitting line through the data points',
   'b': 'To predict the value of a variable based on other variables',
   'c': 'To enable machines to understand and interpret human language',
   'd': 'To estimate the probabilit

In [34]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [35]:
quiz_table_data

[{'MCQ': 'What is the goal of linear regression?',
  'Choices': 'a: To find the best-fitting line through the data points | b: To predict the value of a variable based on other variables | c: To estimate the probability of a categorical outcome | d: To understand and interpret human language',
  'Correct': 'a'},
 {'MCQ': 'When is logistic regression used?',
  'Choices': 'a: When the outcome to predict is categorical | b: When the relationship between variables is linear | c: When processing and analyzing natural language data | d: When generating coherent and contextually relevant text',
  'Correct': 'a'},
 {'MCQ': 'What is the main goal of Natural Language Processing (NLP)?',
  'Choices': 'a: To find the best-fitting line through the data points | b: To predict the value of a variable based on other variables | c: To enable machines to understand and interpret human language | d: To estimate the probability of a categorical outcome',
  'Correct': 'c'},
 {'MCQ': 'What do Large Language

In [36]:
quiz=pd.DataFrame(quiz_table_data)

In [38]:
quiz.to_csv("machinelearning.csv",index=False)